# Imports

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as Layer
import numpy as np
import keras_tuner as kt
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
# from DatasetFormatting import *
import statistics


In [2]:
import re
import pandas as pd

movieFields = ['id', 'title', 'vote_average', 'vote_count', 'genres']
castFields = ['id', 'cast']

movieData = pd.read_csv('movies_metadata.csv', skipinitialspace=True, usecols=movieFields, engine='python')
castData = pd.read_csv('credits.csv', skipinitialspace=True, usecols=castFields, engine='python')

dfMovies = pd.DataFrame(movieData)
dfCast = pd.DataFrame(castData)

df = pd.merge(dfMovies, dfCast, how='inner')


# ---------------------------------------------------------------------------- #
# The following section formats the cast and creates a clean list of actor names.

unformattedNames = df['cast']

# main list of cast grouped by movies
cast_master_copy = []

# list of overall cast by individual names
actor_name_list = []

# reads through the cast string for each movie
for unformattedNameString in unformattedNames:

    # creates new list for current movie
    grouped = ""

    # pulls actors' names out of the paragraph
    line_list = re.findall("(?<=\'name\': )(.*?)(?=,)", unformattedNameString)
    
    # converts list of cast for one movie into a string to manipulate further 
    line_string = str(line_list)

    # separates cast into a unique list per movie
    line_split = line_string.split(", ")

    actor_count = 0
    # reads through each actor per movie
    for name in line_split:
        if (actor_count == 5):
            break
     
        # removes extraneous symbols from the actors' names
        name = name.replace("\"", "")
        name = name.replace("\'", "")
        name = name.replace("[", "")
        name = name.replace("]", "")
        name = name.replace(" ", "")

        # adds actors to two working lists
        # grouped: actors are in lists by movies
        # actor_name_list: puts every actors in one list
        grouped += name + ' '
        actor_name_list.append(name)
        
        actor_count += 1

    # adds formatted cast members to list
    grouped = grouped[:-1]
    cast_master_copy.append(grouped)

# assigns formatted cast to 'cast' column of DataFrame
df['cast'] = cast_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section formats the genre and adds it to the data frame

unformatted_genres = df['genres']

# main list of genres grouped by movies
genre_master_copy = []

# list of overall cast by individual names
genre_name_list = []

# reads through the genre string for each movie
for unformatted_genre_string in unformatted_genres:

    # creates new list for current movie
    genre_grouped = []

    # pulls genres out of the paragraph
    genre_line_list = re.findall("(?<=\'name\': )(.*?)(?=})", unformatted_genre_string)

    # converts list of genres for one movie into a string to manipulate further 
    genre_line_string = str(genre_line_list)

    # separates genre into a unique list per movie
    genre_line_split = genre_line_string.split(", ")

    # reads through each genre per movie & grabs the first one
    for genre in genre_line_split:
     
        # removes extraneous symbols from the genres
        genre = genre.replace("\"", "")
        genre = genre.replace("\'", "")
        genre = genre.replace("[", "")
        genre = genre.replace("]", "")

        genre_str = str(genre)
        break

    # adds formatted genre to list
    genre_master_copy.append(genre_str)

# assigns formatted cast to 'cast' column of DataFrame
df['genres'] = genre_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section appends the genre to the cast string. This preparation is 
# needed for text vectorization / embedding.

final_format_col = []
cast = df['cast'].to_list()
genres = df['genres'].to_list()

i = 0
count = len(cast)
for i in range(count):
    final_string = cast[i] + " " + genres[i]
    final_format_col.append(final_string)

df.insert(loc=0, column='training_col', value=final_format_col)
df.training_col = final_format_col
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section filters out data points that will be harmful for
# our model's accuracy and drops unnecessary columns.

# gets rid of all movies with 0 votes
# Number of movies before operation: 43019
# Number of movies after operation: 22651
df = df.drop(df[df.vote_count < 10].index)
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section handles the creation of the .csv files that will be the
# primary data used to train the models. 

# writes formatted DataFrame values to a new csv
pd.DataFrame.to_csv(df, "formatted_movies.csv", index="false")

# creates a csv of the actor dictionary
# w = csv.writer(open("actor_dict.csv", "w"))
# for key, val in actor_dict.items():
#     w.writerow([key, val])
# ---------------------------------------------------------------------------- #

# Preparing Training Dataset

In [3]:
np.random.seed(1)
tf.random.set_seed(1)

movie_df = df
movie_df.drop(['vote_count'], axis = 1)

data_count = movie_df.shape[0]
training_data_count = int(data_count * 0.75) + 1
test_data_count = data_count - training_data_count

y_train = np.array(movie_df['vote_average'].head(training_data_count))
x_train = np.array(movie_df['training_col'].head(training_data_count))

y_test = np.array(movie_df['vote_average'].tail(test_data_count))
x_test = np.array(movie_df[['training_col']].tail(test_data_count))

std = statistics.stdev(movie_df['vote_average'].to_list())
mean = statistics.mean(movie_df['vote_average'].to_list())
variance = statistics.variance(movie_df['vote_average'].to_list())
mode = statistics.mode(movie_df['vote_average'].to_list())

print(f"STD: {std}")
print(f"mean: {mean}")
print(f"variance: {variance}")
print(f"mode: {mode}")

# print(x_train)
print(f"x_train shape: {x_train.shape}")

STD: 0.9976277124059775
mean: 6.165617412034789
variance: 0.9952610525603838
mode: 6.5
x_train shape: (16989,)


## Text Vectorization For Actors & Genres

In [4]:
VOCAB_SIZE = 201277
encoder = preprocessing.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=6)
features = np.array(df['training_col'])
encoder.adapt(features)

print(np.array(encoder.get_vocabulary()))

['' '[UNK]' 'drama' ... 'aamnashariff' 'aamirbashir' 'aadukalamnaren']


# Recurrent Neural Network

In [6]:
!rm -rf untitled_project

def build_model(hp):

  model = tf.keras.Sequential()
  model.add(encoder)
  
  model.add(Layer.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            output_dim=hp.Int('embedding_neurons', min_value=450, max_value=550, step=50)
            ))
  
  model.add(Layer.SimpleRNN(hp.Int('rnn_neurons', min_value=200, max_value=250, step=50)))

  # for i in range(hp.Int('num_hidden_layers', 1, 2)):
  model.add(Layer.Dense(hp.Int(f'dense{1}_neurons', min_value=60, max_value=80, step=20), activation='relu'))

  model.add(Layer.Dense(1, name='output'))

  optimizer=hp.Choice('optimizer', values=['adam', 'RMSprop'])
  tf.keras.optimizers.Adam(hp.Choice('learning_rate', [.01, .001, .0001]))

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=optimizer,
              metrics=['accuracy'])

  # model.summary()
  return model


tuner = kt.RandomSearch(
build_model,
objective='val_loss',
max_trials=300,  # how many model variations to test?
executions_per_trial=1)  # how many trials per variation? (same model could perform differently

tuner.search_space_summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 6)                 0         
_________________________________________________________________
embedding (Embedding)        (None, 6, 450)            18569250  
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 200)               130200    
_________________________________________________________________
dense (Dense)                (None, 60)                12060     
_________________________________________________________________
output (Dense)               (None, 1)                 61        
Total params: 18,711,571
Trainable params: 18,711,571
Non-trainable params: 0
_________________________________________________________________
Search space summary
Default search space size: 5
embedding_neurons (Int)
{'default': None, 'conditions': [], 

## RNN Training & Validation

In [7]:
tuner.search(x=x_train,
            y=y_train,    
            verbose=2,
            epochs=1,
            batch_size=512,
            callbacks=[],
            validation_data=(x_test, y_test))

tuner.results_summary()

Trial 52 Complete [00h 00m 22s]
val_loss: 1.019692301750183

Best val_loss So Far: 0.7287561297416687
Total elapsed time: 00h 12m 19s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7287561297416687
Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 60
optimizer: adam
learning_rate: 0.0001
Score: 0.7335172891616821
Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7372773289680481
Trial summary
Hyperparameters:
embedding_neurons: 550
rnn_neurons: 250
dense1_neurons: 60
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7380682229995728
Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
op

### Keras-Tuner Hyperparameter Optimization
Go in Editing Mode to View


# Keras Tuner Findings


# Test 1
general test with many values to see how the model reacts to different nums of neurons

### Search Space

Search space summary
Default search space size: 7
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 100, 'max_value': 200, 'step': 50, 'sampling': None}
rnn_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 100, 'max_value': 200, 'step': 50, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 300, 'step': 50, 'sampling': None}
dense2_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 300, 'step': 50, 'sampling': None}
dense3_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 300, 'step': 50, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Results

Trial 300 Complete [00h 00m 07s]

Best val_loss So Far: 0.7231940627098083
Total elapsed time: 00h 30m 58s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')

Trial summary
Hyperparameters:
embedding_neurons: 150
rnn_neurons: 100
dense1_neurons: 50
dense2_neurons: 50
dense3_neurons: 50
optimizer: RMSprop
learning_rate: 0.01
Score: 0.7231940627098083

Trial summary
Hyperparameters:
embedding_neurons: 200
rnn_neurons: 200
dense1_neurons: 50
dense2_neurons: 300
dense3_neurons: 150
optimizer: adam
learning_rate: 0.0001
Score: 0.7882779836654663

Trial summary
Hyperparameters:
embedding_neurons: 150
rnn_neurons: 150
dense1_neurons: 100
dense2_neurons: 50
dense3_neurons: 100
optimizer: adam
learning_rate: 0.0001
Score: 0.7910320162773132

Trial summary
Hyperparameters:
embedding_neurons: 100
rnn_neurons: 150
dense1_neurons: 50
dense2_neurons: 50
dense3_neurons: 200
optimizer: adam
learning_rate: 0.001
Score: 0.7981758117675781

Trial summary
Hyperparameters:
embedding_neurons: 200
rnn_neurons: 150
dense1_neurons: 100
dense2_neurons: 50
dense3_neurons: 50
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8152108192443848

Trial summary
Hyperparameters:
embedding_neurons: 200
rnn_neurons: 200
dense1_neurons: 150
dense2_neurons: 250
dense3_neurons: 300
optimizer: adam
learning_rate: 0.001
Score: 0.8200659155845642

Trial summary
Hyperparameters:
embedding_neurons: 150
rnn_neurons: 200
dense1_neurons: 200
dense2_neurons: 150
dense3_neurons: 250
optimizer: adam
learning_rate: 0.001
Score: 0.8227230906486511

Trial summary
Hyperparameters:
embedding_neurons: 150
rnn_neurons: 200
dense1_neurons: 200
dense2_neurons: 50
dense3_neurons: 200
optimizer: adam
learning_rate: 0.001
Score: 0.8232067227363586

Trial summary
Hyperparameters:
embedding_neurons: 200
rnn_neurons: 150
dense1_neurons: 100
dense2_neurons: 50
dense3_neurons: 200
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8308647871017456

Trial summary
Hyperparameters:
embedding_neurons: 100
rnn_neurons: 200
dense1_neurons: 200
dense2_neurons: 100
dense3_neurons: 300
optimizer: adam
learning_rate: 0.0001
Score: 0.8320332169532776



# Test 2 
mainly focusing on the embedding layers for this test

### Search Space
Search space summary
Default search space size: 5
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 500, 'step': 100, 'sampling': None}
rnn_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 500, 'step': 100, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 50, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Results

Trial 81 Complete [00h 00m 15s]

Best val_loss So Far: 0.6759699583053589
Total elapsed time: 00h 17m 08s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')

Trial summary
Hyperparameters:
embedding_neurons: 400
rnn_neurons: 200
dense1_neurons: 50
optimizer: RMSprop
learning_rate: 0.01
Score: 0.6759699583053589

Trial summary
Hyperparameters:
embedding_neurons: 200
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.01
Score: 0.7111835479736328

Trial summary
Hyperparameters:
embedding_neurons: 200
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.001
Score: 0.7244152426719666

Trial summary
Hyperparameters:
embedding_neurons: 400
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.001
Score: 0.737920880317688

Trial summary
Hyperparameters:
embedding_neurons: 300
rnn_neurons: 300
dense1_neurons: 50
optimizer: adam
learning_rate: 0.001
Score: 0.7473349571228027

Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 400
dense1_neurons: 100
optimizer: adam
learning_rate: 0.0001
Score: 0.7481404542922974

Trial summary
Hyperparameters:
embedding_neurons: 400
rnn_neurons: 200
dense1_neurons: 50
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7580930590629578

Trial summary
Hyperparameters:
embedding_neurons: 300
rnn_neurons: 400
dense1_neurons: 50
optimizer: adam
learning_rate: 0.001
Score: 0.7628225088119507

Trial summary
Hyperparameters:
embedding_neurons: 300
rnn_neurons: 500
dense1_neurons: 50
optimizer: adam
learning_rate: 0.01
Score: 0.7678693532943726

Trial summary
Hyperparameters:
embedding_neurons: 400
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.01
Score: 0.7682520151138306




# Test 3
general test with many values to see how the model reacts to different nums of neurons

### Search Space

Search space summary
Default search space size: 5
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 450, 'max_value': 550, 'step': 50, 'sampling': None}
rnn_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 250, 'step': 50, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 60, 'max_value': 80, 'step': 20, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Results
Trial 52 Complete [00h 00m 22s]
val_loss: 1.019692301750183

Best val_loss So Far: 0.7287561297416687
Total elapsed time: 00h 12m 19s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')

Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7287561297416687

Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 60
optimizer: adam
learning_rate: 0.0001
Score: 0.7335172891616821

Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7372773289680481

Trial summary
Hyperparameters:
embedding_neurons: 550
rnn_neurons: 250
dense1_neurons: 60
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7380682229995728

Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8163751363754272

Trial summary
Hyperparameters:
embedding_neurons: 550
rnn_neurons: 200
dense1_neurons: 80
optimizer: adam
learning_rate: 0.01
Score: 0.8183265924453735

Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 60
optimizer: adam
learning_rate: 0.01
Score: 0.8438754677772522

Trial summary
Hyperparameters:
embedding_neurons: 550
rnn_neurons: 200
dense1_neurons: 60
optimizer: RMSprop
learning_rate: 0.01
Score: 0.8698259592056274

Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 250
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8738608956336975

Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 250
dense1_neurons: 60
optimizer: adam
learning_rate: 0.0001
Score: 0.9030186533927917

